# Machine learning: regression

We'll try to predict missing well logs using regression.

The data are from Colorado. We've already loaded the data into a CSV.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact

%matplotlib inline

In [ ]:
uri = 'https://s3.amazonaws.com/agilegeo/geocomp/Colorado_well_data.csv'

In [ ]:
df = pd.read_csv(uri, index_col=0)

In [ ]:
df.describe()

In [ ]:
df = df.dropna()

# Visual inspection of the data space

In [ ]:
well = 1

features = ['CAL', 'SP', 'GR', 'RES', 'NPHI', 'RHOB']
target = 'DT'

fig, axs = plt.subplots(ncols=len(features)+1, sharey=True, figsize=(8,8))

for ax, feature in zip(axs, features):
    ax.plot(df.loc[df.Well==well, feature], df.loc[df.Well==well, 'Depth'])
    ax.set_title(feature)
axs[-1].plot(df.loc[df.Well==well, target], df.loc[df.Well==well, 'Depth'], color='red')
axs[-1].set_title(target)
axs[-1].invert_yaxis()

In [ ]:
fig, axs = plt.subplots(ncols=len(features), figsize=(15, 3))

for ax, feature in zip(axs, features):
    ax = sns.distplot(df[feature], ax=ax)
    ax.set_title(feature)
    ax.set_yticks([])

Make a 'log<sub>10</sub> resisitivity' to deal with the usual RES distribution:

In [ ]:
df['LogRes'] = np.log10(df.RES)
df = df.loc[df.LogRes > 0]

In [ ]:
sns.distplot(df.LogRes)

And update the `features` list:

In [ ]:
features.remove('RES')
features.append('LogRes')

Now fix the gamma ray:

In [ ]:
df.GR = df.GR.clip(upper=600)

In [ ]:
sns.distplot(df.GR)

And the NPHI:

In [ ]:
df = df.loc[(0 <= df.NPHI) & (df.NPHI <= 0.5)]

In [ ]:
sns.distplot(df.NPHI)

In [ ]:
fig, axs = plt.subplots(ncols=len(features), figsize=(15, 3))

for ax, feature in zip(axs, features):
    ax = sns.distplot(df[feature], ax=ax)
    ax.set_title(feature)
    ax.set_yticks([])

In [ ]:
sns.pairplot(df[df.Well == 1.0], vars=features[1:])

# Split the dataset

In [ ]:
# How many wells are in the data set
df.Well.unique()

In [ ]:
len(df.Well.unique())

 Let's start by training on the first five wells only


In [ ]:
n = 5  # We'll come back and change this number.

In [ ]:
df_train = df[df.Well <= n].copy()
df_val = df[(df.Well >= 70) & (df.Well < 80)].copy()
df_test = df[(df.Well >= 80) & (df.Well < 90)].copy()
df_app = df[df.Well >= 90].copy()

In [ ]:
features = ['GR', 'NPHI', 'RHOB', 'LogRes']
target = 'DT'

In [ ]:
X_train = df_train[features].values
y_train = df_train[target].values

X_val = df_val[features].values
y_val = df_val[target].values

X_test = df_test[features].values
y_test = df_test[target].values

## Check the distributions

In [ ]:
features

In [ ]:
fig, axs = plt.subplots(ncols=len(features), figsize=(15,3))

for ax, feature in zip(axs, features):
    sns.distplot(df_train[feature], ax=ax)
    sns.distplot(df_val[feature], ax=ax)
    sns.distplot(df_test[feature], ax=ax)
    ax.set_yticklabels([])

## Train a model

In [ ]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge

regr = Ridge()
regr.fit(X_train, y_train)
df_val['DT_pred_LR'] = regr.predict(X_val)

In [ ]:
df_val.head()

In [ ]:
def plot_track(df, idx, true, pred):
    fig, ax = plt.subplots(1,1)
    fig.set_size_inches(12,2)
    true = df.loc[df.Well == idx, true]
    pred = df.loc[df.Well == idx, pred]
    depths = df.loc[df.Well == idx, 'Depth']
    ax.plot(depths, true, 'k', lw=1.5)
    ax.plot(depths, pred, 'r', lw=1.5)
    ax.set_xlim(1300, 2400)
    ax.set_ylim(40, 140)
    return

In [ ]:
@interact(idx=(df_val.Well.unique().min(), df_val.Well.unique().max(), 1))
def plot_different_wells(idx):
    plot_track(df_val, idx, 'DT', 'DT_pred_LR')
    return


# Evaluation metrics

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

r2_score(df_val.DT, df_val.DT_pred_LR)

And the RMS error:

In [ ]:
np.sqrt(mean_squared_error(df_val.DT, df_val.DT_pred_LR))

## Check error distribution

In particular, we want to check that:

1. The errors are normally distributed with a zero mean.
1. The variance of the errors is not correlated with the parameters.

There's some good advice about normality tests in [this article by Jason Brownlee](https://machinelearningmastery.com/a-gentle-introduction-to-normality-tests-in-python/).

First we'll just use visual inspection:

In [ ]:
residuals = df_val['DT_pred_LR'] - df_val['DT']

In [ ]:
sns.distplot(residuals)
plt.axvline(0, color='k', lw=0.5)
plt.axvline(residuals.mean(), color='r')
plt.grid(color='k', alpha=0.15)
plt.show()

#### Normality: QQ plot

A quantile-quantile plot generates an idealized distribution, in this case a Gaussian. The idealized samples are divided into quantiles, then each data point in the sample is paired with a similar member from the idealized distribution. The line `'s'` represents the standard 'normal' distribution.

In [ ]:
from statsmodels.graphics.gofplots import qqplot

qqplot(residuals, line='s')
plt.axvline(0, color='k', lw=0.5)
plt.axhline(0, color='k', lw=0.5)
plt.grid(color='k', alpha=0.15)
plt.show()

#### Normality: Shapiro&ndash;Wilk test

Not convinced about this &mdash; seems like most large samples don't fit. `p` just gets very small.

In [ ]:
from scipy.stats import shapiro

res_shuf = residuals.values
np.random.shuffle(res_shuf)

stat, p = shapiro(res_shuf[:500])
print(f'Statistics = {stat:.3f}, p = {p:.3f}')

alpha = 0.05
if p > alpha:
    print('Sample looks Gaussian (fail to reject H0)')
else:
    print('Sample does not look Gaussian (reject H0)')

#### Homoscedasticity: visual inspection

We want to check that the variance of the errors is not correlated with our parameters.

If they are correlated (if the plots below show points with narrow spread at one end and wide at the other), then there are nonlinearities in the data that are not captured by the model. It could be that outliers are skewing the distribution.

In [ ]:
fig, axs = plt.subplots(ncols=len(features), figsize=(16,4), sharey=True)

for ax, feature in zip(axs, features):
    ax.scatter(df_val[feature], residuals, s=1, alpha=0.1)
    ax.set_xlabel(feature)
    ax.axhline(0, color='k', lw=0.5)
    ax.grid(color='k', alpha=0.15)

Seems like there could be an issue in shales (low NPHI), and in rocks with low HC saturation.

## Coefficients

In [ ]:
np.set_printoptions(suppress=True)
regr.coef_

In [ ]:
regr.intercept_

We can makea list of the features that contributed most:

In [ ]:
list(reversed(np.array(features)[np.argsort(np.abs(regr.coef_))]))

In [ ]:
features

In [ ]:
fig, ax = plt.subplots(figsize=(4,8))

idx = np.arange(len(features))
ax.bar(idx, regr.coef_, align='center')
ax.axhline(0, color='k', lw=0.5)
ax.set_xticks(idx)
ax.set_xticklabels(features)
ax.grid(color='k', alpha=0.15)

plt.title('Feature importance')
plt.show()

<html><hr />

<div>
<img src="https://avatars1.githubusercontent.com/u/1692321?s=50"><p style="text-align:center">© Agile Geoscience 2019</p>
</div></html>